# URL 鉴权

## 问题描述

> 1. 现有一个基础服务：用户中心，提供了一系列跟用户信息相关的接口，例如注册、登录、查看用户信息等。
>
> 2. 其他服务可以访问用户中心，但是不同的服务具有不同的权限，权限表如下

```yml
APP_A:
    /user/register
    /user/basic/**
    /user/*/info
    /user/wallet/**/balance
APP_B:
    /user/register
    /user/login
```

> 3. * 匹配任意一个目录，** 匹配任意多个（0 ～ n）子目录

## 问题分析

分成三种匹配情况：精确匹配、前缀匹配、模糊匹配
> 1. 精确匹配：不包含 * 或 ** 的URL。用Hash表
> 2. 前缀匹配：以 ** 结尾的URL。用trie树
> 3. 模糊匹配：其他情况的URL。用回溯算法


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import bisect
from collections import defaultdict
from sortedcontainers import SortedList


class UrlRule:
    class TrieNode:
        def __init__(self, dir=None):
            self.ending = False
            self.dir = dir
            self.children = SortedList(key=lambda trie: trie.dir)

    class Trie:
        def __init__(self):
            self.root = UrlRule.TrieNode()

        def build(self, directory):
            p = self.root
            for d in directory:
                idx = p.children.bisect_key_left(d)
                if idx == len(p.children) or p.children[idx].dir != d:
                    child = UrlRule.TrieNode(d)
                    p.children.add(child)
                p = p.children[idx]
            p.ending = True

        def match(self, url):
            p = self.root
            directory = [d for d in url.split('/') if len(d) > 0]
            for d in directory:
                idx = p.children.bisect_key_left(d)
                if idx == len(p.children) or p.children[idx].dir != d:
                    return False
                elif p.children[idx].ending:
                    return True
                else:
                    p = p.children[idx]
            return False


    def __init__(self):
        self.precise_map = defaultdict(set)
        self.trie = UrlRule.Trie()
        self.fuzzy_map = defaultdict(list)

    def build(self, app, urls):
        for url in urls:
            directory = [d for d in url.split('/') if len(d) > 0]
            if '*' not in directory and '**' not in directory:
                self.precise_map[app].add(url)
            elif directory[-1] == '**':
                self.trie.build(directory[:-1])
            else:
                self.fuzzy_map[app].append(directory)
                
    def _fuzzy_match(self, app, url):
        directory = [d for d in url.split('/') if len(d) > 0]
        
        def dfs(i, j):
            nonlocal match
            if match:
                return
            if i == n and j == m:
                match = True
                return
            elif i == n or j == m:
                return
            
            if d[j] == '*' or directory[i] == d[j]:
                dfs(i+1, j+1)
            elif d[j] == '**':
                for k in range(i, n):
                    dfs(k, j+1)


        for d in self.fuzzy_map[app]:
            i, j, n, m, match = 0, 0, len(directory), len(d), False
            dfs(i, j)
            if match:
                return True
        
        return False

    
    def match(self, app, url):
        if url in self.precise_map[app]:
            return True
        elif self.trie.match(url):
            return True
        else:
            return self._fuzzy_match(app, url)


url_rule = UrlRule()
url_rule.build('APP_A', ['/user/register', '/user/basic/**', '/user/*/info', '/user/wallet/**/balance'])
print('/user/register =', url_rule.match('APP_A', '/user/register'))
print('/user/basic/info/profile = ', url_rule.match('APP_A', '/user/basic/info/profile'))
print('/user/address/info = ', url_rule.match('APP_A', '/user/address/info'))
print('/user/recently-used/address/info = ', url_rule.match('APP_A', '/user/recently-used/address/info'))
print('/user/wallet/balance = ', url_rule.match('APP_A', '/user/wallet/balance'))
print('/user/wallet/cny/balance = ', url_rule.match('APP_A', '/user/wallet/cny/balance'))
print('/user/wallet/cny/saved/balance = ', url_rule.match('APP_A', '/user/wallet/cny/saved/balance'))


/user/register = True
/user/basic/info/profile =  True
/user/address/info =  True
/user/recently-used/address/info =  False
/user/wallet/balance =  True
/user/wallet/cny/balance =  True
/user/wallet/cny/saved/balance =  True
